# Audio Classification Using Edge Impulse
This example uses audio already recorded from fast analog input IN2 and uses local Edge Impulse Classification to classify 5 seconds of audio.

This is based on the Edge Immpulse example:
https://github.com/edgeimpulse/linux-sdk-python/blob/master/examples/audio/classify.py 

The following are required to use the edge_impulse_linux (they can be run from a Jupyter code cell):  
`!apt update`

`!apt install libatlas-base-dev libportaudio2 libportaudiocpp0 portaudio19-dev python3-opencv --yes`

**Reload the Jupyter kernel** then run:

`!pip3 install PyAudio`

`!pip3 install edge_impulse_linux`

## Import Libraries

In [1]:
import os
import sys, getopt
import signal
import numpy as np
from edge_impulse_linux.runner import ImpulseRunner
from scipy.io import wavfile

In [2]:
runner = None
model_file_name = 'modelfile.eim'
audio_file_name = 'float_wave_test_raw.wav'

dir_path = os.path.dirname(os.path.realpath(model_file_name))
modelfile = os.path.join(dir_path, model_file_name)
audio_file = os.path.join(dir_path, audio_file_name)

In [3]:
samplerate, generator = wavfile.read(audio_file)

features = np.array([], dtype=np.int16)
runner = ImpulseRunner(modelfile)

model_info = runner.init()
labels = model_info['model_parameters']['labels']
deploy_version = model_info['project']['deploy_version']
window_size = model_info['model_parameters']['input_features_count']

print('Loaded runner for "' + model_info['project']['owner'] + ' / ' + model_info['project']['name'] + '"')
print('labels: ', labels)
print('deploy_version: ', deploy_version)

Loaded runner for "M0JPI / Sound Test"
labels:  ['background-noise', 'edge-impulse', 'red-pitaya']
deploy_version:  16


In [4]:
samplerate, generator = wavfile.read(audio_file)
features = np.concatenate((features, generator), axis=0)
res = runner.classify(features[:window_size].tolist())

#Show the classification key that has the highest confidence index
best_label = max(res['result']['classification'], key=lambda key: res['result']['classification'][key])
confidence = res['result']['classification'][best_label]
print(best_label,"{:.2%}".format(confidence))

edge-impulse 99.99%


In [5]:
print('Result (%d ms.) ' % (res['timing']['dsp'] + res['timing']['classification']), end='')
for label in labels:
    score = res['result']['classification'][label]
    print('%s: %.2f \t' % (label, score), end='')
    print('', flush=True)

Result (26 ms.) background-noise: 0.00 	
edge-impulse: 1.00 	
red-pitaya: 0.00 	


In [6]:
res

{'result': {'classification': {'background-noise': 5.957508983556181e-05,
   'edge-impulse': 0.9999402761459351,
   'red-pitaya': 9.996507088771978e-08}},
 'timing': {'anomaly': 0,
  'classification': 3,
  'dsp': 23,
  'json': 17,
  'stdin': 36}}